In [2]:
import os
import pandas as pd
import PyPDF2
import Pytesseract
import re



In [3]:
# directory holding the sample of 18 pdfs
samplepdf = '../1.Web Scrapping/PDF'

# capturing all the pdfs
# allpdf = os.listdir(samplepdf)
pdffile = os.listdir(samplepdf)

# displaying total number of pdfs used
totalpdfs = len(pdffile)

print("Total pdfs = ",totalpdfs)

print("PDF names:")
allfiles=[]
for pdf in pdffile:
    allfiles.append(pdf)

for names in allfiles:
    print(names)

Total pdfs =  18
PDF names:
INQ000350057.pdf
INQ000350094.pdf
INQ000350513.pdf
INQ000350691.pdf
INQ000383581.pdf
INQ000383585.pdf
INQ000383998.pdf
INQ000385719.pdf
INQ000395589.pdf
INQ000395913.pdf
INQ000396684.pdf
INQ000396685.pdf
INQ000396686.pdf
INQ000400585.pdf
INQ000412042.pdf
PHT000000014.pdf
PHT000000030.pdf
PHT000000039.pdf


In [4]:
considerfile=[filename for filename in allfiles if filename.startswith("INQ")]


# filtering to use only pdf which has INQ number in filename [Note: these are evidence docs]
for filename in considerfile:
    print(filename)


considerPDFcount=len(considerfile)
# number of pdf documents
print(considerPDFcount)


INQ000350057.pdf
INQ000350094.pdf
INQ000350513.pdf
INQ000350691.pdf
INQ000383581.pdf
INQ000383585.pdf
INQ000383998.pdf
INQ000385719.pdf
INQ000395589.pdf
INQ000395913.pdf
INQ000396684.pdf
INQ000396685.pdf
INQ000396686.pdf
INQ000400585.pdf
INQ000412042.pdf
15


## USING PYPDF2 library

In [4]:
def pydf2extract(pdffiles):

    with open(pdffiles,'rb') as f:
        reader = PyPDF2.PdfReader(f)
    
        results = []

        for page in (reader.pages):
            
            text = page.extract_text()
            if text:
                results.append(text)
    return ' '.join(results) # convert list to a single doc 

textdocspypdf='PYPDF2/PYPDF2docs/'
#if directory is not present, create it
os.makedirs(textdocspypdf, exist_ok=True)

for i in range(0,considerPDFcount):
    try:
        # pdfs with INQ
        filename=considerfile[i]
        print(filename)
        
        # extracting the text from the pdf
        extractedtext = pydf2extract(f'../1.Web Scrapping/PDF/{filename}')
        file = os.path.splitext(filename)[0]
        # creating and storing the result into the directory
        with open(os.path.join(textdocspypdf, f'{file}.txt'), 'w', encoding='utf-8') as f:
            f.write(extractedtext)
    except Exception as e:
        print(f"Exception for PDF {filename} as {e}")


INQ000350057.pdf
INQ000350094.pdf
INQ000350513.pdf
INQ000350691.pdf
INQ000383581.pdf
INQ000383585.pdf
INQ000383998.pdf
INQ000385719.pdf
INQ000395589.pdf
INQ000395913.pdf
INQ000396684.pdf
INQ000396685.pdf
INQ000396686.pdf
INQ000400585.pdf
INQ000412042.pdf


## Converting into Dataframe

In [5]:
import os
import re
import pandas as pd

# function to read and match with csv file
def loadcsv(csvpath):
    df = pd.read_csv(csvpath, delimiter=',') 
    filetitle = {}
    for i, row in df.iterrows():
        filename = row['Title'].split(' - ')[0]  # copy only the first name of the file and match with the full title, appering before '-'
        filetitle[filename] = row['Title']  # match with the correspondong row of that title
    return filetitle

# extraction of date
def extract_date(content):
    datepattern = [
        r'\b\d{1,2}\s+\w+\s+\d{4}\b',      # dates like 21st December 2020
        r'\b\d{1,2}/\d{1,2}/\d{4}\b',      # dates like 22/01/2020
        r'\b\d{1}\s+\w+\s+\d{4}\b'         # dates like 1 December 2020
    ]
    for pattern in datepattern:
        match = re.search(pattern, content)
        if match:
            return match.group(0)
    return "No date found"

# obtaining title, header and date info
def fileinfo(content):
    lines = content.split('\n')
    header = lines[0] if lines else "No header found"  # Get the first line as the header
    date = extract_date(content)
    return header, date

# file path of csv file which contains all the PDF's details from the website 
csvpath = '../1.Web Scrapping/AllFilesDetails.csv'
filetitle = loadcsv(csvpath)

# extracted content file path
textdocscleaned = 'PYPDF2/PYPDF2docs/'
results = []

for i in range(considerPDFcount):
    try:
        filename = considerfile[i]
        basefile = os.path.splitext(filename)[0]
        txtpath = os.path.join(textdocscleaned, f'{basefile}.txt')
        
        if os.path.exists(txtpath):
            with open(txtpath, 'r', encoding='utf-8') as f:
                content = f.read()

            # get title from csv which is store in filetitle
            title = filetitle.get(basefile, "No Title Found")
            header, date = fileinfo(content)

            results.append({
                "Filename": filename,
                "Title": title,
                "Header": header,
                "Date": date
            })
            
            print(f"Filename: {filename}\nTitle: {title}\nHeader: {header}\nDate: {date}\n\n")
        else:
            # if path is not found append 'not exist'
            print(f"Text file {txtpath} does not exist.")
            results.append({
                "Filename": filename,
                "Title": "Not exist",
                "Header": "Not exist",
                "Date": "Not exist"
            })
    # handling exception 
    except Exception as e:
        print(f"Exception for {filename}: {e}")
        results.append({
            "Filename": filename,
            "Title": "Exception",
            "Header": "Exception",
            "Date": str(e)
        })



Filename: INQ000350057.pdf
Title: INQ000350057 - Paper from Welsh Government Technical Advisory Group titled statement regarding NPIs in the pre-christmas period, dated 02/12/2020.
Header: Technical Advisory Group 
Date: 02 December 2020


Filename: INQ000350094.pdf
Title: INQ000350094 - Emails between Lee Waters (Deputy Minister for Economy and Transport) and (Private Secretary Deputy Minister Economy and Transport, Welsh Government) regarding a review of post-firebreak restrictions, between 19/11/2020 and 15/12/2020.
Header: Message 
Date: 15 December 2020


Filename: INQ000350513.pdf
Title: INQ000350513 - Email chain between Chris Whitty (Chief Medical Officer for England, UK Government), Frank Atherton (Chief Medical Officer, Welsh Government), Rob Orford (Chief Scientific Adviser for Health, Welsh Government) and colleagues, regarding an overview of UK Strategy, dated 22/03/2020.
Header: Message 
Date: 22 March 2020


Filename: INQ000350691.pdf
Title: INQ000350691 - Statement by J

In [6]:
# create dataframe and store the content
df = pd.DataFrame(results)
df
df.to_csv('SamplePdfDetail.csv', index=False)


In [7]:
df=pd.read_csv('SamplePdfDetail.csv')
df

,Filename,Title,Header,Date
0,INQ000350057.pdf,INQ000350057 - Paper from Welsh Government Tec...,Technical Advisory Group,02 December 2020
1,INQ000350094.pdf,INQ000350094 - Emails between Lee Waters (Depu...,Message,15 December 2020
2,INQ000350513.pdf,INQ000350513 - Email chain between Chris Whitt...,Message,22 March 2020
3,INQ000350691.pdf,INQ000350691 - Statement by Jane Hutt (Deputy ...,Llywodraeth Cymru,1 December 2020
4,INQ000383581.pdf,INQ000383581 - Minutes of Precautionary SAGE M...,OFFICIAL -SENSITIVE,22 January 2020
5,INQ000383585.pdf,INQ000383585 - Email from DHSC colleague to Si...,Message,31 Jan 2020
6,INQ000383998.pdf,INQ000383998 - Email chain between Dr Rob Orfo...,Message,27 March 2020
7,INQ000385719.pdf,INQ000385719 - Email chain between Rob Orford ...,Message,11 October 2020
8,INQ000395589.pdf,INQ000395589 - Email chain between Gill Richar...,Message,15 April 2020
9,INQ000395913.pdf,INQ000395913 - Email chain between Dr Rob Orfo...,Message,18 October 2020


__Storing Non email documents__

In [10]:
def pydf2extract(pdffiles):

    with open(pdffiles,'rb') as f:
        reader = PyPDF2.PdfReader(f)
    
        results = []

        for page in (reader.pages):
            
            text = page.extract_text()
            if text:
                results.append(text)
    return ' '.join(results) 


# remove whitespaces from the columns
df.columns = df.columns.str.strip()  
df['Header'] = df['Header'].str.strip()  

# filter rows where header is 'Message'
filterdf = df[df['Header'] != 'Message']


textdocspypdf2 = 'PYPDF2/OtherExtraction/'
os.makedirs(textdocspypdf2, exist_ok=True)

for filename in filterdf['Filename']:
    try:
        print(f"{filename}")
        pdfpath = f'../1.Web Scrapping/PDF/{filename}'

        # extract content from pdf
        extractedtext = pydf2extract(pdfpath)

        file = os.path.splitext(filename)[0]
        # storing the extracted text into the directory
        outputpath = os.path.join(textdocspypdf2, f'{file}.txt')
        with open(outputpath, 'w', encoding='utf-8') as f:
            f.write(extractedtext)

    except Exception as e:
        print(f"Exception for PDF {filename}: {e}")


INQ000350057.pdf
INQ000350691.pdf
INQ000383581.pdf
INQ000396684.pdf
INQ000396685.pdf
INQ000396686.pdf
INQ000400585.pdf
INQ000412042.pdf


__Storing Email documents__

In [11]:
def emailextract(filepath):
    # reading the file
    with open(filepath, 'r', encoding='utf-8') as file:
        text = file.read()

    # splitting multiple email into individual
    emails = re.split(r'(?=From: )', text)

    emaildetails = []
    for email in emails:
        if email:
            emaildetails.append(email)
        
    # all the emails splitted are stored in their respective file
    return emaildetails

def directorydata(inputpath, outputpath):
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)

    for filename in os.listdir(inputpath):
        if filename.endswith('.txt'):
            filepath = os.path.join(inputpath, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
                if 'Message' in content:
                    emails = emailextract(filepath)
                    outputfile = os.path.join(outputpath, filename)
                    with open(outputfile, 'w', encoding='utf-8') as text:
                        for i, email in enumerate(emails):
                            text.write(f"{email}\n-------------------------\n\n")
                    print(f"{filename}")

# file path which contains txt files
inputpath = 'PYPDF2/PYPDF2docs/'
# output directory to store the content
outputpath = 'PYPDF2/ExtractedEmails/'
directorydata(inputpath, outputpath)


INQ000350094.txt
INQ000350513.txt
INQ000383585.txt
INQ000383998.txt
INQ000385719.txt
INQ000395589.txt
INQ000395913.txt


__Finding Paragraphs and storing in the directory__

In [12]:
def findparagraphs(text):
    # capture paragraphs with more than 2 lines
    minlines = 2
    
    paragraphs = re.split(r'\n\s*\n', text)
    multiparas = [p for p in paragraphs if len(p.split('\n')) > minlines]
    return multiparas

# both directories to find the paragraphs, OtherExtraction and ExtractedEmails
def storeparagraphs(directories, paradir):
    
    os.makedirs(paradir, exist_ok=True)
    
    for directory in directories:
        # passing each files from the directory 
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                # finding the specific file from the specific directory
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r', encoding='utf-8') as file:
                    txtcontent = file.read()
                # finding and counting the paragraphs
                paragraphs = findparagraphs(txtcontent)
                # storing the paragraphs into separate directory
                outputfile = os.path.join(paradir, filename)
                with open(outputfile, 'w', encoding='utf-8') as text:
                    for para in paragraphs:
                        text.write(para + '\n\n')
                print(f"For file {filename}, {len(paragraphs)} paragraphs are found")

# using both directories
otherextractpath = 'PYPDF2/OtherExtraction/'     # contains all the documents except emails
emailextractpath = 'PYPDF2/ExtractedEmails/'  # contains the documents which are only emails
paradir = 'PYPDF2/CombinedParagraphs/'

# passing the text files from both directories
storeparagraphs([otherextractpath, emailextractpath], paradir)


For file INQ000350057.txt, 1 paragraphs are found
For file INQ000350691.txt, 1 paragraphs are found
For file INQ000383581.txt, 1 paragraphs are found
For file INQ000396684.txt, 1 paragraphs are found
For file INQ000396685.txt, 1 paragraphs are found
For file INQ000396686.txt, 1 paragraphs are found
For file INQ000400585.txt, 1 paragraphs are found
For file INQ000412042.txt, 1 paragraphs are found
For file INQ000350094.txt, 9 paragraphs are found
For file INQ000350513.txt, 3 paragraphs are found
For file INQ000383585.txt, 1 paragraphs are found
For file INQ000383998.txt, 2 paragraphs are found
For file INQ000385719.txt, 3 paragraphs are found
For file INQ000395589.txt, 4 paragraphs are found
For file INQ000395913.txt, 10 paragraphs are found


__Text Cleaning for non emails documents__

In [21]:
df = pd.read_csv('SamplePdfDetail.csv')

# removing white spaces in the 'Header' columns
df['Header'] = df['Header'].str.strip()

# finding only the files which doesn't has in header column, as value 'Message'
filterdf = df[df['Header'] != 'Message']

# input directory which has only paragraphs
paragraphdoc = 'PYPDF2/CombinedParagraphs'
# ouput direcotry which will contain cleaned text
cleanedtextdoc = 'PYPDF2/PYPDF2textclean'

# output directory
os.makedirs(cleanedtextdoc, exist_ok=True)

def cleantxt(text):
    # regex patterns for removing unwanted elements
    patterns = [
        r'OFFICIAL SENSITIVE',
        r'OFFICIAL\s*-\s*SENSITIVE',                          
        r'Llywodraeth Cymru',


        # for handling INQ
        r'I\s*NQ000\w*',                                  # INQ number starting with I followed by NQ000 and numbers/characters
        r'I\s*NQ000\S+',   
        r'I?\s*NQ000\d+_\d+',                             # INQ number starting with "INQ000" and numbers/characters
        r'INQ\s*\d{6,7}_\d{4}',                           # specific footer code with optional spaces
        r'I\s*NQ\s*\d{6,7}_\d{4}',                        # specific footer code with spaces between I, NQ, and numbers
        r'\bINQ\s*\d{6,7}',                               # INQ number without trailing part
        r'I\s*NQ\d{6,7}_\w+',                             # INQ code followed by alphanumeric
        r'I\s*NQ\d{6,7}_\d{4}',                           # I NQ followed by 6 or 7 digits, underscore, and 4 digits
        r'I\s*NQ\d{6,7}_\d+',                             # I NQ followed by 6 or 7 digits, underscore, and any number of digits
        r'I\s*NQ000\d+_\d+',                              # I NQ000 followed by digits, underscore, and any number of digits
        
        # for handling page numbers
        r'Page\s+\d+\s*$',                                # e.g. Page 1, Page 23
        

        # handling emails or links
        r'\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b',     # email addresses
        r'\b(e)\b\s+',                                    # single 'e' with whitespace after
        r'https?://\S+',                                  # URLs starting with http or https
        
        # characters that captured unusual characters
        r'\.{3,}'                                           # removal of ....
        r'|\s?\.\s\.\s\.\s\.',                               # removal of dots which contains whitespaces
        r'|(_{3,}\s?|_\s?_+)',                               # removal of ____
        r'|(-{3,}\s?|-\s?-+)',                               # removal of - - - - 
        r'|■|',                                              # removal of any unusual character
        r'|(\._){1,}',                                       # removal of ._ 
        r'\b\w*(\w)\1{3,}\w*\b'                             # removal of any character that is repeated more than 3 times
        r'\s+(\d)\1{6,}\s+'                                   # removal of any same digits being repeated more than 6 times 

        # patterns to capture unconventional urls
        r'https:[^\s]+',                                  # captured URLs starting with https:
        r'http:[^\s]+',                                   # captured URLs starting with http:
        r'\bhttps?:\S+\s*',                               # capturing URLs which may be differently structured
        r'\bhttps?:\S+(?:\s*\S)*',                        # urls which may have different pattern
        r'\bhttps?:\/\/[a-zA-Z0-9.-]+(\/\S*)?',           # urls with proper structure
        r'\b(?:http|https|ftp):\/\/[a-zA-Z0-9.-]+(?:\/[^\s]*)?', # General URLs
        r'\bhttps?:\/\/\S+(?:\s+\S+)*',                   # urls split by spaces
        r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\s+)+', # Emails split by spaces
        r'\bhttps?:\/\/[^\s]+',                           # general capture for http/https urls
        r'www\.[^\s]+',                                   # urls starting with www
        r'ftp:\/\/[^\s]+',                                # urls starting with ftp

        
    ]

    # applying pattern to the each textual data
    for pattern in patterns:
        # if the pattern for INQ number matches with any of the following, then remove it
        if pattern in [r'INQ\s*\d{6,7}_\d{4}', r'I\s*NQ\s*\d{6,7}_\d{4}', r'\bINQ\s*\d{6,7}', r'I\s*NQ\d{6,7}_\w+']:
            text = re.sub(r'\s*' + pattern + r'\s*', '', text)
        # during extraction process, few documents consider bullet point as 'e'
        elif pattern == r'\b(e)\b\s+':
            # convert single 'e' character back to bullet point
            text = re.sub(pattern, '• ', text)
        elif pattern == r'https?:(?:\s*\S)*':   
            # remove that starts with https or http
            text = re.sub(r'https?://\S+', '', text, flags=re.IGNORECASE | re.MULTILINE)
        else:
            text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.MULTILINE)
            
    # remove any extra white spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# iterate through the filtered DataFrame
for i, row in filterdf.iterrows():
    filename = row['Filename']
    f1name = os.path.splitext(filename)[0]
    txtfilepath = os.path.join(paragraphdoc, f'{f1name}.txt')

    # check if the text file exists
    if os.path.exists(txtfilepath):

        # read the content of the text file
        with open(txtfilepath, 'r', encoding='utf-8') as f:
            text_content = f.read()

        print(f"Original text length for {filename} : {len(text_content)}")  

        # clean the text content
        cleanedtext = cleantxt(text_content)

        print(f"Cleaned text length for {filename} : {len(cleanedtext)}\n\n")  

        # write the cleaned text to a new file in the cleanedtextdocs directory
        cleanedtxtpath = os.path.join(cleanedtextdoc, f'{f1name}.txt')
        with open(cleanedtxtpath, 'w', encoding='utf-8') as f:
            f.write(cleanedtext)

    else:
        print(f"{txtfilepath} does not exist.")


Original text length for INQ000350057.pdf : 29967
Cleaned text length for INQ000350057.pdf : 25996


Original text length for INQ000350691.pdf : 7117
Cleaned text length for INQ000350691.pdf : 6903


Original text length for INQ000383581.pdf : 6607
Cleaned text length for INQ000383581.pdf : 6148


Original text length for INQ000396684.pdf : 26897
Cleaned text length for INQ000396684.pdf : 25926


Original text length for INQ000396685.pdf : 75251
Cleaned text length for INQ000396685.pdf : 73155


Original text length for INQ000396686.pdf : 11803
Cleaned text length for INQ000396686.pdf : 11025


Original text length for INQ000400585.pdf : 34274
Cleaned text length for INQ000400585.pdf : 33189


Original text length for INQ000412042.pdf : 5828
Cleaned text length for INQ000412042.pdf : 4722




__Text Cleaning for Email Extraction__

In [22]:
df = pd.read_csv('SamplePdfDetail.csv')

# removing white spaces in the 'Header' columns
df['Header'] = df['Header'].str.strip()

# finding only the files which have 'Message' in the 'Header' column
filterdf = df[df['Header'] == 'Message']

# combined paragraphs contains only the paragraphs captured 
emailtxt = 'PYPDF2/CombinedParagraphs'
# output directory to store the cleaned text 
cleanedtextdoc = 'PYPDF2/PYPDF2textclean/'

# ensure the output directory exists
os.makedirs(cleanedtextdoc, exist_ok=True)

def emailclean(txt_path):
    # read text file
    with open(txt_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # split the text into individual emails based on the 'From:' keyword
    emails = re.split(r'(?=From: )', text)

    emailinfo = []
    for email in emails:

        emailheader = (
            r'From:.*?\n'  
            r'Sent:.*?\n'  
            # r'To:.*?\n'    # removing this affects some files to lose information
            r'(Cc:.*?\n)?'  
            r'Subject:.*?\n'  
        )
        emailcontent = re.sub(emailheader, '', email, flags=re.DOTALL)
        
        # remove any unncessary content from the emails
        emailcontent = re.sub(r'\S+@\S+', '', emailcontent)  # remove email addresses
        emailcontent = re.sub(r'<[^>]+>', '', emailcontent)  # remove anything in angle brackets
        emailcontent = re.sub(r'(\w+\.\w+@\w+\.\w+)', '', emailcontent)  # remove name and email in the format: name@domain
        
        # for handling INQ
        emailcontent = re.sub(r'I\s*NQ000\w*','',emailcontent)                    # INQ number starting with 'I' followed by NQ000 and numbers/characters
        emailcontent = re.sub(r'I\s*NQ000\S+','',emailcontent)   
        emailcontent = re.sub(r'I?\s*NQ000\d+_\d+','',emailcontent)               # INQ number starting with INQ000 and numbers/characters
        emailcontent = re.sub(r'https?://\S+','',emailcontent)                    # urls starting with http or https
        emailcontent = re.sub(r'INQ\s*\d{6,7}_\d{4}','',emailcontent)             # specific footer code with optional spaces
        emailcontent = re.sub(r'I\s*NQ\s*\d{6,7}_\d{4}','',emailcontent)          # specific footer code with spaces between I NQ and numbers
        emailcontent = re.sub(r'\bINQ\s*\d{6,7}','',emailcontent)                 # INQ number without trailing part
        emailcontent = re.sub(r'I\s*NQ\d{6,7}_\w+','',emailcontent)               # INQ code followed by alphanumeric characters
       

        # remove sequences like ._. ._. ._. ._. ._. ._. ._. ._. ._. ._. ._. 
        emailcontent = re.sub(r'\._\.(\s*\._\.)*', '', emailcontent)
        
        # remove sequences like .-. .-. .-. .-. .-. .-. .-. .-. .-. .-. .-. 
        emailcontent = re.sub(r'\.-\.(\s*\.-\.)*', '', emailcontent)

        # remove 'NR' (assuming case insensitive)
        emailcontent = re.sub(r'\bNR\b', '', emailcontent, flags=re.IGNORECASE)
        
        # remove 'Name Redacted' (assuming case insensitive)
        emailcontent = re.sub(r'\bName\s+Redacted\b', '', emailcontent, flags=re.IGNORECASE)
        emailcontent = re.sub(r'\bNameRedacted\b', '', emailcontent, flags=re.IGNORECASE)
         
        # remove "Ebost/Email"
        emailcontent = re.sub(r'Ebost/Email', '', emailcontent)
        emailcontent = re.sub(r'E-bost', '', emailcontent)
        emailcontent = re.sub(r'E-mail', '', emailcontent)
        emailcontent = re.sub(r'/E-mail', '', emailcontent)
        emailcontent = re.sub(r'/Email', '', emailcontent)
        emailcontent = re.sub(r'E-bost/E-mail', '', emailcontent)

        # remove sequences like ____________________
        emailcontent = re.sub(r'_+', '', emailcontent)

        if emailcontent:
            emailinfo.append(emailcontent)
    return emailinfo

def countwords(text):
    words = re.findall(r'\w+', text)
    return len(words)

# using filetered_df which has the documents that contains only the emails
for i, row in filterdf.iterrows():
    filename = row['Filename']
    f1name = os.path.splitext(filename)[0]
    txtfilepath = os.path.join(emailtxt, f'{f1name}.txt')

    
    if os.path.exists(txtfilepath):
        print(f"{txtfilepath}")

        # cleaning the emails by removing any unncessary data
        emails = emailclean(txtfilepath)

        # prepare the content to be written to the output file
        combinedemail = ""
        for i, email in enumerate(emails):
            # Count words in the email content
            wordcount = countwords(email)
            print(f"Email {i+1} (Word Count: {wordcount}):\n\n")
            
            combinedemail += f"{email}"

        # write all the emails content to a single file in the output directory
        outputfile = os.path.join(cleanedtextdoc, f'{f1name}.txt')
        with open(outputfile, 'w', encoding='utf-8') as f:
            f.write(combinedemail)
    else:
        print(f"Text file {txtfilepath} does not exist.")


PYPDF2/CombinedParagraphs\INQ000350094.txt
Email 1 (Word Count: 39):


Email 2 (Word Count: 158):


Email 3 (Word Count: 44):


Email 4 (Word Count: 13):


Email 5 (Word Count: 66):


Email 6 (Word Count: 79):


Email 7 (Word Count: 573):


Email 8 (Word Count: 561):


Email 9 (Word Count: 280):


PYPDF2/CombinedParagraphs\INQ000350513.txt
Email 1 (Word Count: 349):


Email 2 (Word Count: 15):


Email 3 (Word Count: 143):


PYPDF2/CombinedParagraphs\INQ000383585.txt
Email 1 (Word Count: 426):


PYPDF2/CombinedParagraphs\INQ000383998.txt
Email 1 (Word Count: 34):


Email 2 (Word Count: 586):


PYPDF2/CombinedParagraphs\INQ000385719.txt
Email 1 (Word Count: 0):


Email 2 (Word Count: 439):


Email 3 (Word Count: 69):


PYPDF2/CombinedParagraphs\INQ000395589.txt
Email 1 (Word Count: 203):


Email 2 (Word Count: 114):


Email 3 (Word Count: 532):


Email 4 (Word Count: 1124):


PYPDF2/CombinedParagraphs\INQ000395913.txt
Email 1 (Word Count: 282):


Email 2 (Word Count: 50):


Email 3 (Word